In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import librosa
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import glob
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, Input
from keras import Sequential
import kerastuner as kt
import tensorflow as tf

In [ ]:
def createSpectrogram(path):
  y, sr = librosa.load(path , duration=2.97)
  spect = librosa.feature.melspectrogram(y=y, sr=sr)
  return spect

filename = '../input/gtzan-dataset-music-genre-classification/Data/genres_original'
gen_dict = dict()

data = np.zeros((999, 128, 128)).astype(np.float64)
# labels = np.empty((1000, 1)).astype(np.int32)
labels = []
label_to_num = {k:v for v, k in enumerate(os.listdir(filename))}
num_to_label = {v:k for v, k in enumerate(os.listdir(filename))}

for folder in os.listdir(filename):
    new = filename + "/" + folder + "/*.wav"
    for row, wav_file in enumerate(glob.glob(new)):
        if wav_file != '../input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav':
            spect = createSpectrogram(wav_file)
            data[row, :, :] = spect
            labels.append(label_to_num[folder])
            
labels = np.array(labels)



In [ ]:
labels1 = pd.get_dummies(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2,random_state=42, shuffle = True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5,random_state=42, shuffle = True)



def add_dim(arr):
    shape = arr.shape
    return np.array([elm.reshape(shape[1], shape[2], 1) for elm in arr])

X_train = add_dim(X_train)
X_test = add_dim(X_test)
X_val = add_dim(X_val)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
input_shape = X_train.shape[1:]
def build_CNN_model(input_shape):
    model = Sequential([
    Input(input_shape),
    Conv2D(8,kernel_size=(3,3),strides=(1,1), activation = 'relu'),
    MaxPooling2D((2,2)),
    
    Conv2D(16,kernel_size=(3,3),strides=(1,1), activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32,kernel_size=(3,3),strides=(1,1), activation = 'relu'),
    MaxPooling2D((2,2)),
        
    Conv2D(64,kernel_size=(3,3),strides=(1,1), activation = 'relu'),
    MaxPooling2D((2,2)),

    Conv2D(128,kernel_size=(3,3),strides=(1,1), activation = 'relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dropout(rate=0.3),
    Dense(10, activation='softmax')])

    model.compile(loss='SparseCategoricalCrossentropy',\
                  optimizer='rmsprop', metrics=['accuracy'])
    
    return model

# tuner = kt.Hyperband(build_CNN_model,
#                      objective='val_accuracy',
#                      max_epochs=30,
                     
#                      directory='./',
#                      project_name='pro2')
# tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[stop_early])

# Get the optimal hyperparameters
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Invoking the model and training it
cnn_model = build_CNN_model(input_shape)
cnn_model.fit(X_train, y_train, epochs = 150, validation_data = (X_val, y_val), batch_size = 32, \
             callbacks = [stop_early])


# Evaluating the model
# cnn_model.evaluate((X_test, y_test), batch_size = 10)

In [ ]:
MFCC = pd.read_csv('../input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv')
MFCC_y = MFCC.label
MFCC_X = MFCC.drop(columns = ['label', 'filename', 'length'])

In [ ]:
# MFCC_y_arr = np.delete(np.array(MFCC_y), 554, 0)
# MFCC_X_arr = np.delete(np.array(MFCC_X), 554, 0)
def add_dim(arr):
    shape = arr.shape
    if len(shape) == 3:
        return np.array([elm.reshape(shape[1], shape[2], 1) for elm in arr])
    else:
        return np.array([elm.reshape(shape[1], 1) for elm in arr])
MFCC_X_arr = add_dim(MFCC_X_arr)

In [ ]:
# MFCC_X_train, MFCC_X_test, MFCC_y_train, MFCC_y_test = train_test_split(MFCC_X, MFCC_y, test_size = 0.2,random_state=42, shuffle = True)
# MFCC_X_test, MFCC_X_val, MFCC_y_test, MFCC_y_val = train_test_split(MFCC_X_test, MFCC_y_test, test_size = 0.5,random_state=42, shuffle = True)
def test_split(X, y):
    X = np.array(X)
    y = np.array(y)
    if len(X.shape) == 2:
        X_train = np.empty((800, X.shape[1]))
        X_test = np.empty((100, X.shape[1]))
        X_val = np.empty((99, X.shape[1]))
        y_train = np.empty((800, y.shape[1]))
        y_test = np.empty((100, y.shape[1]))
        y_val = np.empty((99, y.shape[1]))
    else:
        X_train = np.empty((800, X.shape[1], X.shape[2]))
        X_test = np.empty((100, X.shape[1], X.shape[2]))
        X_val = np.empty((99, X.shape[1], X.shape[2]))
        y_train = np.empty((800, y.shape[1]))
        y_test = np.empty((100, y.shape[1]))
        y_val = np.empty((99, y.shape[1]))
    j = 0
    for i in range(0, 1000, 100):
        X_train[j*80: j * 80 + 80,:] = X[i: i +80,:]
        X_test[j*10: j * 10 + 10,:] = X[i + 80: i + 90,:]
        X_val[j*10: j * 10 + 10,:] = X[i + 90: i +100,:]
        y_val[j*10: j * 10 + 10,:] = y[i + 90: i + 100,:]
        y_train[j*80: j * 80 + 80,:] = y[i: i + 80,:]
        y_test[j*10: j * 10 + 10,:] = y[i + 80: i + 90,:]
        j+=1

    return X_train, X_test, X_val, y_train, y_test, y_val
#     return X_train
MFCC_X_train, MFCC_X_test, MFCC_X_val, MFCC_y_train, MFCC_y_test, MFCC_y_val = test_split(MFCC_X_arr, labels1)
Spect_X_train, Spect_X_test, Spect_X_val, Spect_y_train, Spect_y_test, Spect_y_val = test_split(data, labels1)

In [ ]:
class Our_model

In [ ]:
def add_dim_MFCC(arr):
    shape = arr.shape
    return np.array([elm.reshape(shape[1], 1) for elm in arr])

MFCC_X_train = add_dim_MFCC(np.array(MFCC_X_train))
MFCC_X_test = add_dim_MFCC(np.array(MFCC_X_test))
MFCC_X_val = add_dim_MFCC(np.array(MFCC_X_val))

In [ ]:
def my_model(MFCC, Sepct):
    X_sepct_in = Input(MFCC)

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('linear')
  
  else:
    projection=tf.identity(inputs)
    print('none')
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
 
  

 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size,use_bias=True,activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [ ]:


NUM_LAYERS = 4
D_MODEL = data.shape[2]
NUM_HEADS = 4
UNITS = 2048
DROPOUT = 0.1
TIME_STEPS= data.shape[1]
OUTPUT_SIZE=10
EPOCHS = 100
EXPERIMENTS=10



model = transformer(time_steps=TIME_STEPS,
  num_layers=NUM_LAYERS,
  units=UNITS,
  d_model=D_MODEL,
  num_heads=NUM_HEADS,
  dropout=DROPOUT,
  output_size=OUTPUT_SIZE,  
  projection='linear'  )

model.compile(optimizer=tf.keras.optimizers.Adam(0.000001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train,y_train, epochs=EPOCHS, validation_data=(X_val, y_val), verbose = 1)

# accuracy.append(max(history.history['val_accuracy']))

# proj_implemented.append(i)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
file = pd.read_csv("../input/features-extracted-52/Features_Exctracted 52.csv")
file = file.drop(['filename'], axis = 1)
y = file.label
X = file.drop(columns = ['label'])
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X, dtype = float))
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

label_encoder = LabelEncoder()
transformed_train_y = label_encoder.fit_transform(train_y)
transformed_test_y = label_encoder.transform(test_y)
tr_y = pd.get_dummies(train_y)
te_y = pd.get_dummies(test_y)



